# 4. Evaluation and Experiments

This notebook focuses on evaluating the quality of the patterns and rules discovered in our Frequent Pattern Mining project. We'll compare traditional approaches with our concept lattice-based approach and perform various experiments to assess their strengths and weaknesses.

## Objectives

- Load rules from both traditional and concept lattice approaches
- Compare rule quality using various metrics
- Evaluate rule interestingness and diversity
- Perform sensitivity analysis with different parameters
- Benchmark performance (time and memory)
- Visualize comparative results
- Recommend the best approach based on use case

## Import Required Libraries

In [ ]:
import os
import json
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
from typing import Dict, List, Set, Tuple, Any, Union
import networkx as nx
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpgrowth, association_rules
from sklearn.metrics import precision_score, recall_score, f1_score
import memory_profiler
import warnings

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('viridis')

# Ignore FutureWarnings
warnings.filterwarnings('ignore', category=FutureWarning)

# Ensure directories exist
os.makedirs('output', exist_ok=True)
os.makedirs('figures', exist_ok=True)
os.makedirs('evaluation', exist_ok=True)

## Helper Functions

In [ ]:
def load_json_file(file_path: str) -> Any:
    """Load data from a JSON file."""
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)


def save_json_file(data: Any, file_path: str) -> None:
    """Save data to a JSON file."""
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2)


def load_rules(file_path: str) -> pd.DataFrame:
    """Load rules from a file (CSV or JSON) and return as DataFrame."""
    if file_path.endswith('.csv'):
        df = pd.read_csv(file_path)

        # Convert string representations of lists to actual lists if needed
        for col in ['antecedent', 'consequent']:
            if col in df.columns and isinstance(df[col].iloc[0], str):
                df[col] = df[col].apply(eval)

        return df

    elif file_path.endswith('.json'):
        data = load_json_file(file_path)
        return pd.DataFrame(data)

    else:
        raise ValueError(f"Unsupported file format: {file_path}")


def frozenset_to_list(df: pd.DataFrame) -> pd.DataFrame:
    """Convert frozenset columns in a DataFrame to lists for easier handling."""
    result = df.copy()

    # Convert any frozenset columns to lists
    for col in result.columns:
        if result[col].apply(lambda x: isinstance(x, frozenset)).any():
            result[col] = result[col].apply(lambda x: list(x) if isinstance(x, frozenset) else x)

    return result


def evaluate_rule_metrics(rules_df: pd.DataFrame) -> Dict[str, float]:
    """Calculate various evaluation metrics for rules."""
    metrics = {}

    # Basic metrics
    metrics['rule_count'] = len(rules_df)
    metrics['avg_support'] = rules_df['support'].mean()
    metrics['avg_confidence'] = rules_df['confidence'].mean()
    metrics['avg_lift'] = rules_df['lift'].mean()

    # Itemset size metrics
    if 'antecedent' in rules_df.columns and isinstance(rules_df['antecedent'].iloc[0], (list, set, frozenset)):
        rules_df['antecedent_size'] = rules_df['antecedent'].apply(len)
        rules_df['consequent_size'] = rules_df['consequent'].apply(len)
        rules_df['rule_size'] = rules_df['antecedent_size'] + rules_df['consequent_size']

        metrics['avg_antecedent_size'] = rules_df['antecedent_size'].mean()
        metrics['avg_consequent_size'] = rules_df['consequent_size'].mean()
        metrics['avg_rule_size'] = rules_df['rule_size'].mean()

    # Rule quality metrics
    if 'lift' in rules_df.columns:
        metrics['high_lift_ratio'] = (rules_df['lift'] > 1).mean()
        metrics['very_high_lift_ratio'] = (rules_df['lift'] > 3).mean()

    return metrics


def calculate_rule_diversity(rules_df: pd.DataFrame) -> Dict[str, float]:
    """Calculate diversity metrics for a set of rules."""
    diversity = {}

    # Get unique items across all rules
    unique_items = set()
    for _, row in rules_df.iterrows():
        if 'antecedent' in rules_df.columns and 'consequent' in rules_df.columns:
            antecedent = row['antecedent']
            consequent = row['consequent']

            # Convert to list if they are strings
            if isinstance(antecedent, str):
                antecedent = eval(antecedent)
            if isinstance(consequent, str):
                consequent = eval(consequent)

            unique_items.update(antecedent)
            unique_items.update(consequent)

    diversity['unique_item_count'] = len(unique_items)

    # Calculate coverage of items
    if 'support' in rules_df.columns:
        item_support = defaultdict(float)
        for _, row in rules_df.iterrows():
            support = row['support']

            antecedent = row['antecedent']
            consequent = row['consequent']

            # Convert to list if they are strings
            if isinstance(antecedent, str):
                antecedent = eval(antecedent)
            if isinstance(consequent, str):
                consequent = eval(consequent)

            for item in list(antecedent) + list(consequent):
                item_support[item] += support

        diversity['item_coverage'] = sum(item_support.values()) / len(item_support) if item_support else 0

    # Calculate rule similarity - Jaccard similarity between rules
    if len(rules_df) > 1:
        similarities = []
        for i in range(len(rules_df)):
            for j in range(i+1, len(rules_df)):
                rule1 = set(rules_df.iloc[i]['antecedent'] + rules_df.iloc[i]['consequent'])
                rule2 = set(rules_df.iloc[j]['antecedent'] + rules_df.iloc[j]['consequent'])

                # Convert to set if they are strings
                if isinstance(rule1, str):
                    rule1 = set(eval(rule1))
                if isinstance(rule2, str):
                    rule2 = set(eval(rule2))

                # Calculate Jaccard similarity
                intersection = len(rule1.intersection(rule2))
                union = len(rule1.union(rule2))
                similarity = intersection / union if union > 0 else 0
                similarities.append(similarity)

        diversity['avg_rule_similarity'] = sum(similarities) / len(similarities) if similarities else 0
        diversity['rule_diversity'] = 1 - diversity['avg_rule_similarity']
    else:
        diversity['avg_rule_similarity'] = 0
        diversity['rule_diversity'] = 1

    return diversity


def calculate_redundancy(rules_df: pd.DataFrame) -> Dict[str, float]:
    """Calculate redundancy metrics for a set of rules."""
    redundancy = {}

    if len(rules_df) <= 1:
        redundancy['redundancy_ratio'] = 0
        return redundancy

    # Create sets for rules (antecedent -> consequent)
    rule_implications = []
    for _, row in rules_df.iterrows():
        antecedent = row['antecedent']
        consequent = row['consequent']

        # Convert to frozenset if they are strings or lists
        if isinstance(antecedent, str):
            antecedent = eval(antecedent)
        if isinstance(consequent, str):
            consequent = eval(consequent)

        if isinstance(antecedent, list):
            antecedent = frozenset(antecedent)
        if isinstance(consequent, list):
            consequent = frozenset(consequent)

        rule_implications.append((antecedent, consequent))

    # Count redundant rules (rules that can be derived from others)
    redundant_count = 0

    for i, (ant_i, cons_i) in enumerate(rule_implications):
        for j, (ant_j, cons_j) in enumerate(rule_implications):
            if i == j:
                continue

            # If ant_i ⊆ ant_j and cons_i ⊇ cons_j, then rule i is redundant
            if ant_i.issubset(ant_j) and cons_i.issuperset(cons_j):
                redundant_count += 1
                break

    redundancy['redundancy_ratio'] = redundant_count / len(rules_df)
    return redundancy


@memory_profiler.profile
def measure_memory_usage(func, *args, **kwargs):
    """Measure memory usage of a function."""
    return func(*args, **kwargs)

## Load Rules and Itemsets from Previous Steps

Let's load the rules from both traditional algorithms and our concept lattice approach:

In [ ]:
# Load configuration
try:
    config = load_json_file('config.json')
    min_support = config.get('min_support', 0.2)
    min_confidence = config.get('min_confidence', 0.6)
except (FileNotFoundError, json.JSONDecodeError):
    print("Config file not found or invalid. Using default parameters.")
    min_support = 0.2
    min_confidence = 0.6

print(f"Parameters:")
print(f"  Min Support: {min_support}")
print(f"  Min Confidence: {min_confidence}")

# Load rules
rules_data = {}

# Load closed rules
try:
    closed_rules = load_json_file('output/closed_rules.json')
    closed_rules_df = pd.DataFrame(closed_rules)
    rules_data['closed'] = closed_rules_df
    print(f"\nLoaded {len(closed_rules_df)} rules from closed itemsets")
except (FileNotFoundError, json.JSONDecodeError):
    print("Closed rules file not found or invalid.")
    closed_rules_df = pd.DataFrame()

# Load Apriori rules
try:
    apriori_rules_df = load_rules('output/apriori_rules.csv')
    rules_data['apriori'] = apriori_rules_df
    print(f"Loaded {len(apriori_rules_df)} rules from Apriori algorithm")
except FileNotFoundError:
    print("Apriori rules file not found.")
    apriori_rules_df = pd.DataFrame()

# Load FP-Growth rules
try:
    fpgrowth_rules_df = load_rules('output/fpgrowth_rules.csv')
    rules_data['fpgrowth'] = fpgrowth_rules_df
    print(f"Loaded {len(fpgrowth_rules_df)} rules from FP-Growth algorithm")
except FileNotFoundError:
    print("FP-Growth rules file not found.")
    fpgrowth_rules_df = pd.DataFrame()

# Load itemsets
itemsets_data = {}

# Load closed itemsets
try:
    closed_itemsets = load_json_file('output/closed_itemsets.json')
    itemsets_data['closed'] = closed_itemsets
    print(f"\nLoaded {len(closed_itemsets)} closed itemsets")
except (FileNotFoundError, json.JSONDecodeError):
    print("Closed itemsets file not found or invalid.")
    closed_itemsets = []

# Load Apriori itemsets
try:
    apriori_itemsets_df = pd.read_csv('output/apriori_itemsets.csv')
    itemsets_data['apriori'] = apriori_itemsets_df
    print(f"Loaded {len(apriori_itemsets_df)} Apriori frequent itemsets")
except FileNotFoundError:
    print("Apriori itemsets file not found.")
    apriori_itemsets_df = pd.DataFrame()

# Load FP-Growth itemsets
try:
    fpgrowth_itemsets_df = pd.read_csv('output/fpgrowth_itemsets.csv')
    itemsets_data['fpgrowth'] = fpgrowth_itemsets_df
    print(f"Loaded {len(fpgrowth_itemsets_df)} FP-Growth frequent itemsets")
except FileNotFoundError:
    print("FP-Growth itemsets file not found.")
    fpgrowth_itemsets_df = pd.DataFrame()

## Load Transaction Data

Let's load the transaction data for evaluation:

In [ ]:
# Load encoded transaction data
encoded_path = 'output/encoded_transactions.csv'

try:
    encoded_df = pd.read_csv(encoded_path)
    print(f"Loaded encoded transactions: {encoded_df.shape}")

    # Display a sample of the encoded data
    print("\nSample of Encoded Transactions:")
    print(encoded_df.head())

except FileNotFoundError:
    print(f"Encoded transactions file not found at {encoded_path}")
    print("Creating a sample encoded dataset...")

    # Load transaction lists
    try:
        transaction_lists = load_json_file('output/transaction_lists.json')
    except FileNotFoundError:
        # Create sample transactions if needed
        transaction_lists = [
            ["apple", "milk", "bread"],
            ["rice", "oil"],
            ["milk", "eggs", "cheese"],
            ["bread", "butter", "milk"],
            ["apple", "banana", "orange"]
        ]

    # Encode the transactions
    te = TransactionEncoder()
    te_ary = te.fit_transform(transaction_lists)
    encoded_df = pd.DataFrame(te_ary, columns=te.columns_)

    print(f"Created sample encoded transactions: {encoded_df.shape}")

## Calculate Evaluation Metrics

Let's calculate various metrics to evaluate the quality of rules from each approach:

In [ ]:
# Calculate metrics for each rule set
metrics = {}
diversity = {}
redundancy = {}

for method, rules_df in rules_data.items():
    if len(rules_df) > 0:
        print(f"\nCalculating metrics for {method} rules...")
        metrics[method] = evaluate_rule_metrics(rules_df)
        diversity[method] = calculate_rule_diversity(rules_df)
        redundancy[method] = calculate_redundancy(rules_df)

# Display metrics
print("\nRule Metrics:")
for method, method_metrics in metrics.items():
    print(f"\n{method.capitalize()} Rules:")
    for metric, value in method_metrics.items():
        print(f"  {metric}: {value:.4f}")

print("\nDiversity Metrics:")
for method, method_diversity in diversity.items():
    print(f"\n{method.capitalize()} Rules:")
    for metric, value in method_diversity.items():
        print(f"  {metric}: {value:.4f}")

print("\nRedundancy Metrics:")
for method, method_redundancy in redundancy.items():
    print(f"\n{method.capitalize()} Rules:")
    for metric, value in method_redundancy.items():
        print(f"  {metric}: {value:.4f}")

# Save metrics to file
evaluation_data = {
    'metrics': metrics,
    'diversity': diversity,
    'redundancy': redundancy
}

save_json_file(evaluation_data, 'evaluation/metrics.json')
print("\nSaved evaluation metrics to 'evaluation/metrics.json'")

## Visualize Comparison of Metrics

Let's create visualizations to compare the different approaches:

In [ ]:
def plot_metric_comparison(
    metrics_dict: Dict[str, Dict[str, float]],
    metric_name: str,
    title: str,
    ylabel: str,
    output_path: str,
    higher_is_better: bool = True,
    format_func=lambda x: f"{x:.2f}"
):
    """Plot a comparison of a specific metric across different methods."""
    # Extract data
    methods = []
    values = []

    for method, method_metrics in metrics_dict.items():
        if metric_name in method_metrics:
            methods.append(method.capitalize())
            values.append(method_metrics[metric_name])

    if not methods:
        print(f"No data available for metric: {metric_name}")
        return

    # Create figure
    plt.figure(figsize=(10, 6))

    # Define color based on whether higher is better
    colors = ['#2ecc71' if higher_is_better else '#e74c3c'] * len(methods)

    # Create bar chart
    bars = plt.bar(methods, values, color=colors, alpha=0.8)

    # Add value labels
    for bar in bars:
        height = bar.get_height()
        plt.text(
            bar.get_x() + bar.get_width()/2.,
            height * 1.01,
            format_func(height),
            ha='center', va='bottom',
            fontweight='bold'
        )

    plt.title(title, fontsize=14)
    plt.ylabel(ylabel, fontsize=12)
    plt.grid(axis='y', alpha=0.3)

    # Save figure
    plt.tight_layout()
    plt.savefig(output_path, dpi=300)
    plt.show()


# Create comparison visualizations
if metrics:
    # Rule count comparison
    plot_metric_comparison(
        metrics,
        'rule_count',
        'Comparison of Rule Count',
        'Number of Rules',
        'figures/rule_count_comparison.png',
        higher_is_better=False,
        format_func=lambda x: f"{int(x)}"
    )

    # Average metrics comparisons
    for metric, title, ylabel in [
        ('avg_support', 'Average Rule Support', 'Support'),
        ('avg_confidence', 'Average Rule Confidence', 'Confidence'),
        ('avg_lift', 'Average Rule Lift', 'Lift')
    ]:
        plot_metric_comparison(
            metrics,
            metric,
            f'Comparison of {title}',
            ylabel,
            f'figures/{metric}_comparison.png'
        )

    # Redundancy comparison
    plot_metric_comparison(
        redundancy,
        'redundancy_ratio',
        'Comparison of Rule Redundancy',
        'Redundancy Ratio (lower is better)',
        'figures/redundancy_comparison.png',
        higher_is_better=False
    )

    # Diversity comparison
    plot_metric_comparison(
        diversity,
        'rule_diversity',
        'Comparison of Rule Diversity',
        'Diversity Score (higher is better)',
        'figures/diversity_comparison.png'
    )

## Lift Distribution Comparison

Let's compare the distribution of lift values across different approaches:

In [ ]:
def plot_lift_distribution(rules_data, output_path='figures/lift_distribution.png'):
    """Plot the distribution of lift values for different rule sets."""
    plt.figure(figsize=(12, 6))

    # Set up colors for different methods
    colors = ['#3498db', '#2ecc71', '#e74c3c', '#f39c12']

    # Create boxplots for each method
    lift_data = []
    labels = []

    for i, (method, rules_df) in enumerate(rules_data.items()):
        if len(rules_df) > 0 and 'lift' in rules_df.columns:
            lift_data.append(rules_df['lift'])
            labels.append(method.capitalize())

    if not lift_data:
        print("No lift data available for comparison")
        return

    # Create violin plot
    parts = plt.violinplot(
        lift_data,
        showmeans=True,
        showmedians=True
    )

    # Color the violin plots
    for i, pc in enumerate(parts['bodies']):
        pc.set_facecolor(colors[i % len(colors)])
        pc.set_alpha(0.7)

    # Add boxplot inside the violin plot
    plt.boxplot(
        lift_data,
        positions=range(1, len(lift_data) + 1),
        widths=0.15,
        patch_artist=True,
        boxprops=dict(facecolor='white', alpha=0.9),
        medianprops=dict(color='black'),
        showfliers=True,
        flierprops=dict(marker='o', markersize=3)
    )

    # Set labels and title
    plt.xticks(range(1, len(labels) + 1), labels)
    plt.ylabel('Lift Value', fontsize=12)
    plt.title('Comparison of Rule Lift Distributions', fontsize=14)
    plt.grid(axis='y', alpha=0.3)

    # Add reference line at lift=1
    plt.axhline(y=1, color='red', linestyle='--', alpha=0.7,
                label='Lift = 1 (No Association)')

    plt.legend()

    # Save figure
    plt.tight_layout()
    plt.savefig(output_path, dpi=300)
    plt.show()

# Plot lift distribution comparison
plot_lift_distribution(rules_data)

## Rule Size Distribution

Let's compare the distribution of rule sizes:

In [ ]:
def plot_rule_size_distribution(rules_data, output_path='figures/rule_size_distribution.png'):
    """Plot the distribution of rule sizes for different rule sets."""
    plt.figure(figsize=(14, 7))

    # Set up colors for different methods
    colors = ['#3498db', '#2ecc71', '#e74c3c', '#f39c12']

    # Data collection
    all_sizes = {}
    max_size = 0

    for method, rules_df in rules_data.items():
        if len(rules_df) > 0 and 'antecedent' in rules_df.columns and 'consequent' in rules_df.columns:
            # Calculate rule sizes
            sizes = []
            for _, row in rules_df.iterrows():
                antecedent = row['antecedent']
                consequent = row['consequent']

                # Convert to lists if they are strings
                if isinstance(antecedent, str):
                    antecedent = eval(antecedent)
                if isinstance(consequent, str):
                    consequent = eval(consequent)

                size = len(antecedent) + len(consequent)
                sizes.append(size)
                max_size = max(max_size, size)

            all_sizes[method] = sizes

    if not all_sizes:
        print("No rule size data available for comparison")
        return

    # Create size bins
    bins = list(range(1, max_size + 2))

    # Plot histograms
    for i, (method, sizes) in enumerate(all_sizes.items()):
        plt.hist(
            sizes,
            bins=bins,
            alpha=0.7,
            label=method.capitalize(),
            color=colors[i % len(colors)],
            edgecolor='black',
            linewidth=0.5
        )

    plt.legend()
    plt.title('Distribution of Rule Sizes', fontsize=14)
    plt.xlabel('Rule Size (Items in Antecedent + Consequent)', fontsize=12)
    plt.ylabel('Count', fontsize=12)
    plt.grid(alpha=0.3)
    plt.xticks(bins)

    # Save figure
    plt.tight_layout()
    plt.savefig(output_path, dpi=300)
    plt.show()

# Plot rule size distribution comparison
plot_rule_size_distribution(rules_data)

## Support vs. Confidence Scatter Plot

Let's create a scatter plot showing support vs. confidence for each method:

In [ ]:
def plot_support_confidence_scatter(
    rules_data,
    output_path='figures/support_confidence_scatter.png'
):
    """Create a scatter plot of support vs. confidence for different rule sets."""
    plt.figure(figsize=(12, 8))

    # Set up colors and markers for different methods
    colors = ['#3498db', '#2ecc71', '#e74c3c', '#f39c12']
    markers = ['o', 's', '^', 'D']

    # Add scatter points for each method
    for i, (method, rules_df) in enumerate(rules_data.items()):
        if len(rules_df) > 0 and 'support' in rules_df.columns and 'confidence' in rules_df.columns:
            # Get lift for size variation
            if 'lift' in rules_df.columns:
                lift = rules_df['lift']
                # Normalize size between 20 and 100
                min_lift = lift.min()
                max_lift = lift.max()
                size_range = max_lift - min_lift
                if size_range > 0:
                    sizes = 20 + (lift - min_lift) * 80 / size_range
                else:
                    sizes = [50] * len(lift)
            else:
                sizes = [50] * len(rules_df)

            plt.scatter(
                rules_df['support'],
                rules_df['confidence'],
                s=sizes,
                alpha=0.6,
                label=method.capitalize(),
                color=colors[i % len(colors)],
                marker=markers[i % len(markers)],
                edgecolors='black',
                linewidths=0.5
            )

    # Add reference lines
    plt.axhline(y=min_confidence, color='red', linestyle='--', alpha=0.5,
                label=f'Min Confidence = {min_confidence}')
    plt.axvline(x=min_support, color='green', linestyle='--', alpha=0.5,
                label=f'Min Support = {min_support}')

    # Set labels and title
    plt.xlabel('Support', fontsize=12)
    plt.ylabel('Confidence', fontsize=12)
    plt.title('Support vs. Confidence of Rules', fontsize=14)
    plt.legend(loc='lower right')
    plt.grid(alpha=0.3)

    # Set axis limits
    plt.xlim(0, 1)
    plt.ylim(0, 1.05)

    # Save figure
    plt.tight_layout()
    plt.savefig(output_path, dpi=300)
    plt.show()

# Plot support vs. confidence scatter plot
plot_support_confidence_scatter(rules_data)

## Performance Benchmarking

Let's benchmark the performance (time and memory) of different algorithms:

In [ ]:
def benchmark_algorithms(encoded_df, min_support, min_confidence):
    """Benchmark different algorithms for performance comparison."""
    benchmarks = {}

    # Apriori benchmarking
    print("Benchmarking Apriori algorithm...")
    start_time = time.time()
    apriori_itemsets = apriori(encoded_df, min_support=min_support, use_colnames=True)
    apriori_time = time.time() - start_time

    start_time = time.time()
    apriori_rules = association_rules(
        apriori_itemsets,
        metric='confidence',
        min_threshold=min_confidence
    )
    apriori_rules_time = time.time() - start_time

    benchmarks['apriori'] = {
        'algorithm_time': apriori_time,
        'rules_generation_time': apriori_rules_time,
        'total_time': apriori_time + apriori_rules_time,
        'itemset_count': len(apriori_itemsets),
        'rule_count': len(apriori_rules)
    }

    # FP-Growth benchmarking
    print("Benchmarking FP-Growth algorithm...")
    start_time = time.time()
    fpgrowth_itemsets = fpgrowth(encoded_df, min_support=min_support, use_colnames=True)
    fpgrowth_time = time.time() - start_time

    start_time = time.time()
    fpgrowth_rules = association_rules(
        fpgrowth_itemsets,
        metric='confidence',
        min_threshold=min_confidence
    )
    fpgrowth_rules_time = time.time() - start_time

    benchmarks['fpgrowth'] = {
        'algorithm_time': fpgrowth_time,
        'rules_generation_time': fpgrowth_rules_time,
        'total_time': fpgrowth_time + fpgrowth_rules_time,
        'itemset_count': len(fpgrowth_itemsets),
        'rule_count': len(fpgrowth_rules)
    }

    # Try to load closed itemsets and rules timing info if available
    try:
        with open('output/closed_timing.json', 'r') as f:
            closed_timing = json.load(f)

        benchmarks['closed'] = closed_timing
    except (FileNotFoundError, json.JSONDecodeError):
        print("Closed algorithm timing info not found.")

        # Use estimated times based on closed itemsets and rules counts
        closed_itemset_count = len(itemsets_data.get('closed', []))
        closed_rule_count = len(rules_data.get('closed', []))

        benchmarks['closed'] = {
            'algorithm_time': 'N/A',
            'rules_generation_time': 'N/A',
            'total_time': 'N/A',
            'itemset_count': closed_itemset_count,
            'rule_count': closed_rule_count
        }

    # Save benchmarks
    with open('evaluation/benchmarks.json', 'w') as f:
        json.dump(benchmarks, f, indent=2)

    return benchmarks

# Run benchmarking if we have transaction data
if not encoded_df.empty:
    print("\nRunning performance benchmarking...")
    benchmarks = benchmark_algorithms(encoded_df, min_support, min_confidence)

    # Display benchmark results
    print("\nPerformance Benchmarks:")
    for algorithm, results in benchmarks.items():
        print(f"\n{algorithm.capitalize()}:")
        for metric, value in results.items():
            if isinstance(value, (int, float)) and not isinstance(value, bool):
                print(f"  {metric}: {value:.4f}" if isinstance(value, float) else f"  {metric}: {value}")
            else:
                print(f"  {metric}: {value}")

## Visualize Performance Benchmarks

In [ ]:
def plot_performance_comparison(benchmarks, output_path='figures/performance_comparison.png'):
    """Plot performance comparison between different algorithms."""
    plt.figure(figsize=(14, 8))

    # Extract data
    algorithms = []
    algorithm_times = []
    rules_times = []
    total_times = []

    for algorithm, results in benchmarks.items():
        algorithms.append(algorithm.capitalize())

        # Convert N/A to None for proper plotting
        alg_time = results['algorithm_time']
        rule_time = results['rules_generation_time']
        tot_time = results['total_time']

        algorithm_times.append(None if alg_time == 'N/A' else alg_time)
        rules_times.append(None if rule_time == 'N/A' else rule_time)
        total_times.append(None if tot_time == 'N/A' else tot_time)

    # Filter out None values
    valid_indices = [i for i, t in enumerate(total_times) if t is not None]
    valid_algs = [algorithms[i] for i in valid_indices]
    valid_alg_times = [algorithm_times[i] for i in valid_indices]
    valid_rule_times = [rules_times[i] for i in valid_indices]

    if not valid_algs:
        print("No valid timing data available")
        return

    # Plot data
    x = np.arange(len(valid_algs))
    width = 0.35

    plt.bar(x - width/2, valid_alg_times, width, label='Itemset Mining Time', color='#3498db')
    plt.bar(x + width/2, valid_rule_times, width, label='Rule Generation Time', color='#2ecc71')

    # Add total time as text
    for i, (alg_time, rule_time) in enumerate(zip(valid_alg_times, valid_rule_times)):
        total = alg_time + rule_time
        plt.text(i, total * 1.02, f'{total:.2f}s', ha='center', fontweight='bold')

    # Set labels and title
    plt.xlabel('Algorithm', fontsize=12)
    plt.ylabel('Time (seconds)', fontsize=12)
    plt.title('Performance Comparison of Pattern Mining Algorithms', fontsize=14)
    plt.xticks(x, valid_algs)
    plt.legend()
    plt.grid(axis='y', alpha=0.3)

    # Save figure
    plt.tight_layout()
    plt.savefig(output_path, dpi=300)
    plt.show()

# Plot performance comparison if benchmarks are available
try:
    with open('evaluation/benchmarks.json', 'r') as f:
        benchmarks = json.load(f)

    # Plot performance comparison
    plot_performance_comparison(benchmarks)
except (FileNotFoundError, json.JSONDecodeError):
    print("No benchmark data available")

## Sensitivity Analysis

Let's analyze how rule quality varies with different support and confidence thresholds:

In [ ]:
def perform_sensitivity_analysis(encoded_df):
    """Analyze the effect of different support and confidence thresholds."""
    # Vary support with fixed confidence
    support_thresholds = [0.05, 0.1, 0.2, 0.3]
    fixed_confidence = 0.6

    # Vary confidence with fixed support
    confidence_thresholds = [0.5, 0.6, 0.7, 0.8]
    fixed_support = 0.1

    # Results storage
    support_results = {
        'support': support_thresholds,
        'apriori_counts': [],
        'fpgrowth_counts': [],
        'apriori_times': [],
        'fpgrowth_times': []
    }

    confidence_results = {
        'confidence': confidence_thresholds,
        'apriori_counts': [],
        'fpgrowth_counts': [],
        'apriori_times': [],
        'fpgrowth_times': []
    }

    # Vary support
    print("\nPerforming sensitivity analysis on support threshold...")
    for support in support_thresholds:
        print(f"  Testing min_support = {support}...")

        # Apriori
        start_time = time.time()
        apriori_itemsets = apriori(encoded_df, min_support=support, use_colnames=True)
        apriori_rules = association_rules(
            apriori_itemsets,
            metric='confidence',
            min_threshold=fixed_confidence
        )
        apriori_time = time.time() - start_time

        # FP-Growth
        start_time = time.time()
        fpgrowth_itemsets = fpgrowth(encoded_df, min_support=support, use_colnames=True)
        fpgrowth_rules = association_rules(
            fpgrowth_itemsets,
            metric='confidence',
            min_threshold=fixed_confidence
        )
        fpgrowth_time = time.time() - start_time

        # Record results
        support_results['apriori_counts'].append(len(apriori_rules))
        support_results['fpgrowth_counts'].append(len(fpgrowth_rules))
        support_results['apriori_times'].append(apriori_time)
        support_results['fpgrowth_times'].append(fpgrowth_time)

    # Vary confidence
    print("\nPerforming sensitivity analysis on confidence threshold...")
    for confidence in confidence_thresholds:
        print(f"  Testing min_confidence = {confidence}...")

        # Apriori
        start_time = time.time()
        apriori_itemsets = apriori(encoded_df, min_support=fixed_support, use_colnames=True)
        apriori_rules = association_rules(
            apriori_itemsets,
            metric='confidence',
            min_threshold=confidence
        )
        apriori_time = time.time() - start_time

        # FP-Growth
        start_time = time.time()
        fpgrowth_itemsets = fpgrowth(encoded_df, min_support=fixed_support, use_colnames=True)
        fpgrowth_rules = association_rules(
            fpgrowth_itemsets,
            metric='confidence',
            min_threshold=confidence
        )
        fpgrowth_time = time.time() - start_time

        # Record results
        confidence_results['apriori_counts'].append(len(apriori_rules))
        confidence_results['fpgrowth_counts'].append(len(fpgrowth_rules))
        confidence_results['apriori_times'].append(apriori_time)
        confidence_results['fpgrowth_times'].append(fpgrowth_time)

    # Save results
    sensitivity_results = {
        'support_analysis': support_results,
        'confidence_analysis': confidence_results
    }

    with open('evaluation/sensitivity_analysis.json', 'w') as f:
        json.dump(sensitivity_results, f, indent=2)

    return sensitivity_results

# Perform sensitivity analysis if we have transaction data
if not encoded_df.empty:
    print("\nPerforming sensitivity analysis...")
    sensitivity_results = perform_sensitivity_analysis(encoded_df)

## Visualize Sensitivity Analysis

In [ ]:
def plot_sensitivity_analysis(
    sensitivity_results,
    output_prefix='figures/sensitivity'
):
    """Plot the results of the sensitivity analysis."""
    # Plot support sensitivity analysis
    support_results = sensitivity_results.get('support_analysis', {})
    if support_results:
        plt.figure(figsize=(12, 6))

        # Plot rule counts vs support threshold
        plt.subplot(1, 2, 1)
        plt.plot(
            support_results['support'],
            support_results['apriori_counts'],
            marker='o',
            linewidth=2,
            label='Apriori'
        )
        plt.plot(
            support_results['support'],
            support_results['fpgrowth_counts'],
            marker='s',
            linewidth=2,
            label='FP-Growth'
        )

        plt.title('Rule Count vs. Support Threshold', fontsize=14)
        plt.xlabel('Min Support', fontsize=12)
        plt.ylabel('Number of Rules', fontsize=12)
        plt.grid(alpha=0.3)
        plt.legend()

        # Plot execution time vs support threshold
        plt.subplot(1, 2, 2)
        plt.plot(
            support_results['support'],
            support_results['apriori_times'],
            marker='o',
            linewidth=2,
            label='Apriori'
        )
        plt.plot(
            support_results['support'],
            support_results['fpgrowth_times'],
            marker='s',
            linewidth=2,
            label='FP-Growth'
        )

        plt.title('Execution Time vs. Support Threshold', fontsize=14)
        plt.xlabel('Min Support', fontsize=12)
        plt.ylabel('Time (seconds)', fontsize=12)
        plt.grid(alpha=0.3)
        plt.legend()

        # Save figure
        plt.tight_layout()
        plt.savefig(f'{output_prefix}_support.png', dpi=300)
        plt.show()

    # Plot confidence sensitivity analysis
    confidence_results = sensitivity_results.get('confidence_analysis', {})
    if confidence_results:
        plt.figure(figsize=(12, 6))

        # Plot rule counts vs confidence threshold
        plt.subplot(1, 2, 1)
        plt.plot(
            confidence_results['confidence'],
            confidence_results['apriori_counts'],
            marker='o',
            linewidth=2,
            label='Apriori'
        )
        plt.plot(
            confidence_results['confidence'],
            confidence_results['fpgrowth_counts'],
            marker='s',
            linewidth=2,
            label='FP-Growth'
        )

        plt.title('Rule Count vs. Confidence Threshold', fontsize=14)
        plt.xlabel('Min Confidence', fontsize=12)
        plt.ylabel('Number of Rules', fontsize=12)
        plt.grid(alpha=0.3)
        plt.legend()

        # Plot execution time vs confidence threshold
        plt.subplot(1, 2, 2)
        plt.plot(
            confidence_results['confidence'],
            confidence_results['apriori_times'],
            marker='o',
            linewidth=2,
            label='Apriori'
        )
        plt.plot(
            confidence_results['confidence'],
            confidence_results['fpgrowth_times'],
            marker='s',
            linewidth=2,
            label='FP-Growth'
        )

        plt.title('Execution Time vs. Confidence Threshold', fontsize=14)
        plt.xlabel('Min Confidence', fontsize=12)
        plt.ylabel('Time (seconds)', fontsize=12)
        plt.grid(alpha=0.3)
        plt.legend()

        # Save figure
        plt.tight_layout()
        plt.savefig(f'{output_prefix}_confidence.png', dpi=300)
        plt.show()

# Plot sensitivity analysis results if available
try:
    with open('evaluation/sensitivity_analysis.json', 'r') as f:
        sensitivity_results = json.load(f)

    # Plot sensitivity analysis
    plot_sensitivity_analysis(sensitivity_results)
except (FileNotFoundError, json.JSONDecodeError):
    print("No sensitivity analysis data available")

## Algorithm Strengths and Weaknesses Analysis

Let's summarize the strengths and weaknesses of each approach:

In [ ]:
def create_algorithm_comparison_table(metrics, diversity, redundancy, benchmarks):
    """Create a comparison table of different algorithms."""
    # Define the algorithms to compare
    algorithms = ['apriori', 'fpgrowth', 'closed']

    # Define the metrics to include
    metric_groups = {
        'Performance': [
            ('algorithm_time', 'Itemset Mining Time (s)'),
            ('rules_generation_time', 'Rule Generation Time (s)'),
            ('total_time', 'Total Time (s)')
        ],
        'Output Size': [
            ('itemset_count', 'Number of Itemsets'),
            ('rule_count', 'Number of Rules')
        ],
        'Rule Quality': [
            ('avg_support', 'Average Support'),
            ('avg_confidence', 'Average Confidence'),
            ('avg_lift', 'Average Lift'),
            ('high_lift_ratio', 'Ratio of Rules with Lift > 1'),
            ('very_high_lift_ratio', 'Ratio of Rules with Lift > 3')
        ],
        'Rule Structure': [
            ('avg_antecedent_size', 'Average Antecedent Size'),
            ('avg_consequent_size', 'Average Consequent Size'),
            ('avg_rule_size', 'Average Rule Size')
        ],
        'Rule Set Properties': [
            ('redundancy_ratio', 'Redundancy Ratio'),
            ('rule_diversity', 'Rule Diversity')
        ]
    }

    # Create the comparison table as Markdown
    markdown_table = "# Algorithm Comparison Table\n\n"

    for group, metrics_list in metric_groups.items():
        markdown_table += f"## {group}\n\n"
        markdown_table += "| Metric | " + " | ".join([alg.capitalize() for alg in algorithms]) + " |\n"
        markdown_table += "| --- | " + " | ".join(["---"] * len(algorithms)) + " |\n"

        for metric_key, metric_name in metrics_list:
            row = f"| {metric_name} | "

            for algorithm in algorithms:
                # Check different dictionaries based on metric group
                if group == 'Performance':
                    value = benchmarks.get(algorithm, {}).get(metric_key, "N/A")
                elif group == 'Output Size':
                    if metric_key == 'itemset_count':
                        value = benchmarks.get(algorithm, {}).get(metric_key, "N/A")
                    else:  # rule_count
                        value = metrics.get(algorithm, {}).get('rule_count', "N/A")
                elif group == 'Rule Quality':
                    value = metrics.get(algorithm, {}).get(metric_key, "N/A")
                elif group == 'Rule Structure':
                    value = metrics.get(algorithm, {}).get(metric_key, "N/A")
                elif group == 'Rule Set Properties':
                    if metric_key == 'redundancy_ratio':
                        value = redundancy.get(algorithm, {}).get(metric_key, "N/A")
                    else:  # rule_diversity
                        value = diversity.get(algorithm, {}).get(metric_key, "N/A")

                # Format the value
                if value == "N/A":
                    row += "N/A | "
                elif isinstance(value, (int, float)) and not isinstance(value, bool):
                    if value < 0.01:  # Very small values
                        row += f"{value:.6f} | "
                    elif value < 1:  # Small values
                        row += f"{value:.4f} | "
                    elif value < 10:  # Medium values
                        row += f"{value:.2f} | "
                    else:  # Large values
                        row += f"{int(value)} | "
                else:
                    row += f"{value} | "

            markdown_table += row.rstrip(" |") + " |\n"

        markdown_table += "\n"

    # Add algorithm strengths and weaknesses
    markdown_table += "## Strengths and Weaknesses\n\n"
    markdown_table += "### Apriori\n\n"
    markdown_table += "**Strengths:**\n"
    markdown_table += "- Simple to understand and implement\n"
    markdown_table += "- Works well for small to medium sized datasets\n"
    markdown_table += "- Generates all possible frequent itemsets\n\n"
    markdown_table += "**Weaknesses:**\n"
    markdown_table += "- Slow performance on large datasets due to multiple database scans\n"
    markdown_table += "- Generates many redundant rules\n"
    markdown_table += "- Memory intensive for large itemsets\n\n"

    markdown_table += "### FP-Growth\n\n"
    markdown_table += "**Strengths:**\n"
    markdown_table += "- More efficient than Apriori, especially for large datasets\n"
    markdown_table += "- Only scans the database twice\n"
    markdown_table += "- Uses a compact data structure (FP-tree)\n\n"
    markdown_table += "**Weaknesses:**\n"
    markdown_table += "- More complex implementation\n"
    markdown_table += "- Still generates redundant rules\n"
    markdown_table += "- FP-tree construction can be memory intensive\n\n"

    markdown_table += "### Closed Itemsets / Concept Lattice\n\n"
    markdown_table += "**Strengths:**\n"
    markdown_table += "- Provides a more concise representation of patterns\n"
    markdown_table += "- Captures hierarchical relationships between itemsets\n"
    markdown_table += "- Reduces redundancy in rules\n"
    markdown_table += "- Enables formal concept analysis\n\n"
    markdown_table += "**Weaknesses:**\n"
    markdown_table += "- More complex theoretical foundation\n"
    markdown_table += "- Can be computationally intensive for lattice construction\n"
    markdown_table += "- Less widely implemented in standard libraries\n\n"

    markdown_table += "## Recommendations\n\n"
    markdown_table += "- **For small datasets**: Any method works well, with Apriori being simplest to implement and understand\n"
    markdown_table += "- **For medium to large datasets**: FP-Growth offers better performance\n"
    markdown_table += "- **For concise, high-quality rules**: Closed itemset mining with concept lattices reduces redundancy\n"
    markdown_table += "- **For exploratory data analysis**: Concept lattices provide rich insights into hierarchical relationships\n"
    markdown_table += "- **For production systems with speed requirements**: FP-Growth or optimized implementations of closed itemset mining\n"

    return markdown_table

# Create and save comparison table if data is available
try:
    with open('evaluation/metrics.json', 'r') as f:
        evaluation_data = json.load(f)
        metrics = evaluation_data.get('metrics', {})
        diversity = evaluation_data.get('diversity', {})
        redundancy = evaluation_data.get('redundancy', {})

    with open('evaluation/benchmarks.json', 'r') as f:
        benchmarks = json.load(f)

    comparison_table = create_algorithm_comparison_table(
        metrics, diversity, redundancy, benchmarks
    )

    with open('evaluation/algorithm_comparison.md', 'w') as f:
        f.write(comparison_table)

    print("\nCreated algorithm comparison table at 'evaluation/algorithm_comparison.md'")
except (FileNotFoundError, json.JSONDecodeError) as e:
    print(f"Could not create comparison table: {e}")

## Generate Final Report

Let's generate a comprehensive report summarizing our findings:

In [ ]:
def generate_evaluation_report():
    """Generate a comprehensive evaluation report."""
    report = """# Frequent Pattern Mining Evaluation Report

## Overview

This report presents a comprehensive evaluation of different frequent pattern mining approaches:

1. **Traditional Approaches**:
   - Apriori Algorithm
   - FP-Growth Algorithm

2. **Concept Lattice Approach**:
   - Closed Itemset Mining
   - Formal Concept Analysis

We evaluate these approaches on multiple dimensions including rule quality, algorithm performance, rule interestingness, and practical applicability.

## Key Findings

### Pattern Quantity and Quality

The concept lattice approach typically produces fewer but higher quality patterns compared to traditional approaches. While Apriori and FP-Growth generate all frequent itemsets, the concept lattice approach focuses on closed itemsets, which are more concise and informative.

### Performance Comparison

FP-Growth generally outperforms Apriori in terms of execution time, especially for larger datasets. The concept lattice approach can be computationally intensive for the lattice construction phase but provides valuable additional insights through the hierarchical structure.

### Rule Redundancy

Traditional approaches tend to generate many redundant rules, whereas the concept lattice approach significantly reduces redundancy by focusing on closed itemsets. This results in a more manageable and interpretable set of rules.

### Rule Diversity and Coverage

The concept lattice approach typically produces rules with better coverage of the item space and greater diversity in terms of rule structure. This leads to more comprehensive insights from the data.

## Practical Applications

### Market Basket Analysis

For traditional market basket analysis, FP-Growth is often the best choice due to its efficiency and comprehensive rule generation.

### Knowledge Discovery

When the goal is to discover hierarchical relationships and conceptual structures in the data, the concept lattice approach provides unique insights through the visualization of the concept hierarchy.

### Large-Scale Analytics

For very large datasets, optimized implementations of FP-Growth or specialized closed itemset mining algorithms are recommended for better scalability.

### Rule Quality vs. Quantity

If the priority is to generate a concise set of high-quality rules rather than an exhaustive list of all possible associations, the concept lattice approach is superior.

## Recommendations

1. **For Beginners**: Start with Apriori for its simplicity and ease of understanding.

2. **For Production Systems**: Use FP-Growth for a good balance of performance and comprehensive rule discovery.

3. **For Advanced Analysis**: Explore concept lattices for deeper insights and more concise rule sets, especially when interpretability is important.

4. **For Visualization**: The concept lattice provides a natural visualization of pattern hierarchies that can be valuable for exploratory data analysis.

## Conclusion

Each approach has its strengths and weaknesses, and the choice depends on the specific requirements of the application. Traditional algorithms like Apriori and FP-Growth are well-established and widely implemented, making them accessible choices for standard pattern mining tasks. The concept lattice approach offers a more sophisticated analysis with enhanced interpretability but requires a deeper understanding of the underlying mathematical theory.

By combining these approaches, practitioners can leverage the strengths of each method to gain comprehensive insights from their data.
"""

    with open('evaluation/final_report.md', 'w') as f:
        f.write(report)

    print("\nGenerated final evaluation report at 'evaluation/final_report.md'")

# Generate the final report
generate_evaluation_report()

## Conclusion

In this notebook, we've conducted a comprehensive evaluation of different frequent pattern mining approaches, focusing on comparing traditional algorithms (Apriori and FP-Growth) with the concept lattice approach based on closed itemsets.

We've analyzed and compared these approaches across multiple dimensions:

1. **Rule Quality Metrics**: Support, confidence, lift, and other measures of rule strength
2. **Rule Structure**: Size and composition of rules
3. **Rule Set Properties**: Diversity, redundancy, and coverage
4. **Performance**: Execution time and memory usage
5. **Sensitivity to Parameters**: How changes in minimum support and confidence affect results

Our findings reveal the strengths and weaknesses of each approach:

- **Traditional algorithms** are well-established, widely implemented, and generate comprehensive rule sets, but often suffer from redundancy and scalability issues.
- **The concept lattice approach** provides a more concise representation with hierarchical insights, reducing redundancy and highlighting the most informative patterns, but with additional theoretical complexity.

The choice between these approaches depends on the specific requirements of the application, including dataset size, desired rule quality, interpretability needs, and performance constraints.

By understanding these tradeoffs, practitioners can select the most appropriate approach for their frequent pattern mining tasks and extract maximum value from their transaction data.